# PROJECT:

In [9]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable
from biogeme import models
from biogeme import results as res

In [10]:
df = pd.read_table('lpmc11.dat')
df['cost_driving'] = df['cost_driving_fuel'] + df['cost_driving_ccharge']
database = db.Database('lpmc', df)

# Model 0

**Variables:**

In [11]:
dur_walking = Variable('dur_walking') #time for walking
dur_cycling = Variable('dur_cycling') #time for cycling

dur_pt_access = Variable('dur_pt_access')
dur_pt_rail = Variable('dur_pt_rail')
dur_pt_bus = Variable('dur_pt_bus')
dur_pt_int = Variable('dur_pt_int')

dur_pt = dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int #total time for public transport
dur_driving = Variable('dur_driving')

cost_transit = Variable('cost_transit') #cost for public transport
# cost_driving_fuel = Variable('cost_driving_fuel')
# cost_driving_ccharge = Variable('cost_driving_ccharge')

cost_driving = Variable('cost_driving') #total cost for driving

travel_mode = Variable('travel_mode') #Choice

**Parameters:**

In [12]:
constant_walking = Beta('constant_walking', 0, None, None, 0)
constant_cycling = Beta('constant_cycling', 0, None, None, 0)
constant_pt = Beta('constant_pt', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

In [13]:
v_walk_Model0 = (
    constant_walking
    + beta_time * dur_walking
)

v_cycling_Model0 = (
    constant_cycling
    + beta_time * dur_cycling
)

v_PT_Model0 = (
    constant_pt
    + beta_cost * cost_transit
    + beta_time * dur_pt
)

v_driving_Model0 = (
    beta_cost * cost_driving
    + beta_time * dur_driving
)

In [14]:
V_Model0 = {1: v_walk_Model0, 2: v_cycling_Model0, 3: v_PT_Model0, 4: v_driving_Model0}
logprob_Model0 = models.loglogit(V_Model0, None, travel_mode)
biogeme_Model0 = bio.BIOGEME(database, logprob_Model0) #create the biogeme estimator
biogeme_Model0.modelName = 'logit_Model0'
res_Model0 = biogeme_Model0.estimate()

In [15]:
print(res_Model0.printGeneralStatistics())

Number of estimated parameters:	5
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4676.875
Final log likelihood:	-4676.875
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00107
Akaike Information Criterion:	9363.751
Bayesian Information Criterion:	9396.337
Final gradient norm:	2.6771E-04
Nbr of threads:	64



In [16]:
res_Model0.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost,-0.156585,0.012667,-12.361233,0.0
beta_time,-5.226955,0.191905,-27.237207,0.0
constant_cycling,-2.485329,0.086252,-28.814839,0.0
constant_pt,0.726783,0.047157,15.411971,0.0
constant_walking,1.187012,0.078799,15.063738,0.0


The sign of the 2 generic coefficent leaves us satisfied. We know it's reasonable that utility is decreased when cost and/or travel time increase. It also seems that time impacts more the utility, since in asbolute value beta_time is higher than beta_cost. Concernig the ASCs, we normalized with respect to ASC_car so this is our baseline. We notice that ASC_walking is the highest so it seems that people, assuiming the same atrributes values, would choose this alternative. Anyway we see that, since the value of beta_time is high in absoulute value, and walking usually requires more time than the others, it will be penalized, as we expected. Cycling seems really disadvantaged, since its ASC is already low, and it has the same beta_time (due to generic coefficent assumption). Significance of value is confirmed by the t-test values that are all above the 1.96 threshold

# Model 1

We decide to split beta_time into 4 alternative specific parameters. Our choice is due to the fact that this coefficent is present in every utility function, and we think it can undergo a wider variation, since time perception is strictly related to the chosen travel mode.

In [17]:
constant_walking = Beta('constant_walking', 0, None, None, 0)
constant_cycling = Beta('constant_cycling', 0, None, None, 0)
constant_pt = Beta('constant_pt', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time_walking = Beta('beta_time_walking', 0, None, None, 0)
beta_time_cycling = Beta('beta_time_cycling', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_driving = Beta('beta_time_driving', 0, None, None, 0)

In [18]:
v_walk_Model1 = (
    constant_walking
    + beta_time_walking * dur_walking
)

v_cycling_Model1 = (
    constant_cycling
    + beta_time_cycling * dur_cycling
)

v_PT_Model1 = (
    constant_pt
    + beta_cost * cost_transit
    + beta_time_pt * dur_pt
)

v_driving_Model1 = (
    beta_cost * cost_driving
    + beta_time_driving * dur_driving
)

In [19]:
V_Model1 = {1: v_walk_Model1, 2: v_cycling_Model1, 3: v_PT_Model1, 4: v_driving_Model1}
logprob_Model1 = models.loglogit(V_Model1, None, travel_mode)
biogeme_Model1 = bio.BIOGEME(database, logprob_Model1)
biogeme_Model1.modelName = 'logit_Model1'
res_Model1 = biogeme_Model1.estimate()

In [20]:
print(res_Model1.printGeneralStatistics())

Number of estimated parameters:	8
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4372.911
Final log likelihood:	-4372.911
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00183
Akaike Information Criterion:	8761.822
Bayesian Information Criterion:	8813.959
Final gradient norm:	2.8918E-03
Nbr of threads:	64



In [21]:
res_Model1.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost,-0.136269,0.013923,-9.787478,0.000000e+00
beta_time_cycling,-5.436177,0.492925,-11.028412,0.000000e+00
beta_time_driving,-5.794675,0.384636,-15.065358,0.000000e+00
beta_time_pt,-3.127377,0.243891,-12.822864,0.000000e+00
beta_time_walking,-8.161128,0.386290,-21.126960,0.000000e+00
constant_cycling,-2.560688,0.150700,-16.991966,0.000000e+00
constant_pt,-0.405922,0.069445,-5.845207,5.059401e-09
constant_walking,1.933359,0.129143,14.970653,0.000000e+00


As before, signs are reasonable. Concernign the values of the coefficents, we firstly notice that beta_cost reamined pretty much the same and it seems reasonable becuase people usually care how much money they spend rather than for what they do spend. The wide range of values we can see on beta_time strenghtens our initial assumption, in fact we see how beta_walking is really low, since walking requires phisycal effort, while the two alternatives that just require the individual's focus (driving and cycling) are pretty similar and this seems fine. In the end we see that beta_time_PT is instead the lowest in absolute value, and this is logic for us since when you are in public transport you are able to do other activities or just realx without having to mantain focus. Again, as before, ASC-walking is still bigger than car, as cycling is still the lowest. The only change we notice is the sign of ASC_pt, which is probably due to the value of the beta_time_pt we previously analyzed. Significance of value is confirmed by the t-test values that are all above the 1.96 threshold

In [22]:
res_Model1.likelihood_ratio_test(res_Model0, 0.01)

LRTuple(message='H0 can be rejected at level 1.0%', statistic=607.9289925585272, threshold=11.344866730144373)

To compare the 2 models we used the likelihood ratio test. This is doable since model 0 is a linear restriction of model 1 (assuming all the alternative specific beta_time equal). Our null-hypotesis is that the resricted model (in this case model 0) is the true model. We set the significance level of the test to 0,01 and as we can see from the result the null-hypotesis is rejected, hence our preferred model is model 1

# Model 2:

- Interaction of age with ASCs:

We select 'age' as socio-economic characteristic since we think it has a stornger correlation with the choice of a transport mode rather than sex, which is the other socio-ecomic characteristic we found in the dataset. In fact, if we assume two individuals have the same age, from our experience, their choice (putting all the other parametres equal) would not probably be different. On the other hand, age can impact the choice both from an availability point of view (i.e. young or really old may not have a driving license) and a physical effort one.

In [23]:
age = Variable('age')

In [24]:
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time_walking = Beta('beta_time_walking', 0, None, None, 0)
beta_time_cycling = Beta('beta_time_cycling', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_driving = Beta('beta_time_driving', 0, None, None, 0)
ASC_w = Beta('ASC_w', 0, None, None, 0)
ASC_c = Beta('ASC_c', 0, None, None, 0)
ASC_pt = Beta('ASC_pt', 0, None, None, 0)


In [25]:
interacted_ASC_w = ASC_w * age/99
interacted_ASC_c = ASC_c * age/99
interacted_ASC_pt = ASC_pt * age/99


In [26]:
v_w_Model2_ASC = (
    interacted_ASC_w
    + beta_time_walking * dur_walking
)

v_c_Model2_ASC = (
    interacted_ASC_c
    + beta_time_cycling * dur_cycling
)

v_pt_Model2_ASC = (
    interacted_ASC_pt
    + beta_cost * cost_transit
    + beta_time_pt * dur_pt
)

v_driving_Model2_ASC = (
    beta_cost * cost_driving
    + beta_time_driving * dur_driving
)

In [27]:
V_Model2_ASC = {1: v_w_Model2_ASC , 2: v_c_Model2_ASC, 3: v_pt_Model2_ASC, 4: v_driving_Model2_ASC}
logprob_Model2_ASC = models.loglogit(V_Model2_ASC, None, travel_mode)
biogeme_Model2_ASC = bio.BIOGEME(database, logprob_Model2_ASC)
biogeme_Model2_ASC.modelName = 'logit_Model2_ASC'
res_Model2_ASC = biogeme_Model2_ASC.estimate()

In [28]:
print(res_Model2_ASC.printGeneralStatistics())

Number of estimated parameters:	8
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4615.576
Final log likelihood:	-4615.576
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00173
Akaike Information Criterion:	9247.151
Bayesian Information Criterion:	9299.289
Final gradient norm:	4.2811E-03
Nbr of threads:	64



In [29]:
res_Model2_ASC.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_c,-4.876934,0.374371,-13.027024,0.000000e+00
ASC_pt,-0.998334,0.133146,-7.498069,6.483702e-14
ASC_w,1.531976,0.158595,9.659663,0.000000e+00
beta_cost,-0.150734,0.014566,-10.348652,0.000000e+00
beta_time_cycling,-8.105935,0.665889,-12.173110,0.000000e+00
beta_time_driving,-5.751921,0.391627,-14.687258,0.000000e+00
beta_time_pt,-3.123986,0.244501,-12.776987,0.000000e+00
beta_time_walking,-5.265470,0.197674,-26.637139,0.000000e+00


The estimated signs leaves us satisfied. Moreover, from this estimation we can notice that walking is still the highest among the ASCs, but it is also a lot penalized by its beta_time, which we could expect. Anyway the fact that beta_time_cycling is so low seems pretty strange to us, since we thinik that time has a bigger impact in other contexts such as walking. We can also see that beta_cost has a less signifcant impact on the choices.

- Interaction of age with cost:

When we have to analyze the interaction of the age with one of the variables, we decided to link it with cost, since we think that indviduals may have a differtent perception of money depending on their age. For example, when you're a student you give a different value to money than the one you give them when you've been working for several years.

In [30]:
constant_walking = Beta('constant_walking', 0, None, None, 0)
constant_cycling = Beta('constant_cycling', 0, None, None, 0)
constant_pt = Beta('constant_pt', 0, None, None, 0)
beta_time_walking = Beta('beta_time_walking', 0, None, None, 0)
beta_time_cycling = Beta('beta_time_cycling', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_driving = Beta('beta_time_driving', 0, None, None, 0)
beta_cost_int = Beta('beta_cost_int', 0, None, None, 0)

In [31]:
beta_cost_interacted = beta_cost_int * age/99

In [32]:
v_w_Model2_cost = (
    constant_walking
    + beta_time_walking * dur_walking
)

v_c_Model2_cost = (
    constant_cycling
    + beta_time_cycling * dur_cycling
)

v_pt_Model2_cost= (
    constant_pt
    + beta_cost_interacted * cost_transit    
    + beta_time_pt * dur_pt
)

v_driving_Model2_cost = (
    beta_cost_interacted * cost_driving
    + beta_time_driving * dur_driving
)

In [33]:
V_Model2_cost = {1: v_w_Model2_cost, 2: v_c_Model2_cost, 3: v_pt_Model2_cost, 4: v_driving_Model2_cost}
logprob_Model2_cost = models.loglogit(V_Model2_cost, None, travel_mode)
biogeme_Model2_cost = bio.BIOGEME(database, logprob_Model2_cost)
biogeme_Model2_cost.modelName = 'logit_Model2_cost'
res_Model2_cost = biogeme_Model2_cost.estimate()

In [34]:
print(res_Model2_cost.printGeneralStatistics())

Number of estimated parameters:	8
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4387.311
Final log likelihood:	-4387.311
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00182
Akaike Information Criterion:	8790.622
Bayesian Information Criterion:	8842.76
Final gradient norm:	1.8462E-03
Nbr of threads:	64



In [35]:
res_Model2_cost.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost_int,-0.287265,0.039548,-7.263774,3.763656e-13
beta_time_cycling,-5.430248,0.494069,-10.990875,0.000000e+00
beta_time_driving,-5.958971,0.382057,-15.597075,0.000000e+00
beta_time_pt,-3.218752,0.242768,-13.258575,0.000000e+00
beta_time_walking,-8.180582,0.386817,-21.148430,0.000000e+00
constant_cycling,-2.566973,0.150922,-17.008640,0.000000e+00
constant_pt,-0.407455,0.069331,-5.876925,4.179564e-09
constant_walking,1.939261,0.129410,14.985393,0.000000e+00


The signs we get are still ok for us, then we can also see that this time beta_cost is lower than before, and this is due to the fact that we are considering the interaction with age. We can also see that the ASCs are still pretty much the same. 

Now it's the tome of testing the models. Since the previous preferred model is Model1, and none of our last two models can be seen as a restriction of Model 1, we cannot apply loglikelihood ratio test directly. We instead want to perfrom a Cox test, where we define a third model, as the sum of model 1 and the interaction term. We then perform the likelihood ratio test between the model 1 and the cox model and then between the interacted model and the cox model. The null hypotesis for each of this test is that the restricted model is the better one. We set the significance level of the test to 0,05. If the cox_model results to be the best in both case we have to look for better models, if the models are not rejected in both case we need to change our test and in the other cases we'll be able to state the preferred model. Of course this has to be done for both the interactions we defined. 

In [36]:
v_w_cox1 = v_walk_Model1 + interacted_ASC_w
v_c_cox1 = v_cycling_Model1 + interacted_ASC_c
v_pt_cox1 = v_PT_Model1 + interacted_ASC_pt
v_driving_cox1 = v_driving_Model1

In [37]:
V_cox1 = {1: v_w_cox1, 2: v_c_cox1, 3: v_pt_cox1, 4: v_driving_cox1}
logprob_cox1 = models.loglogit(V_cox1, None, travel_mode)
biogeme_cox1 = bio.BIOGEME(database, logprob_cox1)
biogeme_cox1.modelName = 'logit_cox1'
res_cox1 = biogeme_cox1.estimate()

In [38]:
print(res_cox1.printGeneralStatistics())

Number of estimated parameters:	11
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4348.347
Final log likelihood:	-4348.347
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00253
Akaike Information Criterion:	8718.694
Bayesian Information Criterion:	8790.383
Final gradient norm:	2.3308E-03
Nbr of threads:	64



In [39]:
res_cox1.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_c,-1.056331,0.365441,-2.890563,3.845519e-03
ASC_pt,-0.959524,0.187752,-5.110597,3.211423e-07
ASC_w,-1.397502,0.236492,-5.909296,3.435729e-09
beta_cost,-0.137210,0.013975,-9.818290,0.000000e+00
beta_time_cycling,-5.425166,0.493299,-10.997715,0.000000e+00
beta_time_driving,-5.742581,0.383501,-14.974113,0.000000e+00
beta_time_pt,-3.113525,0.243493,-12.786929,0.000000e+00
beta_time_walking,-8.186674,0.389671,-21.009213,0.000000e+00
constant_cycling,-2.131890,0.210885,-10.109264,0.000000e+00
constant_pt,-0.012516,0.102428,-0.122195,9.027448e-01


In [40]:
print(res_cox1.likelihood_ratio_test(res_Model1, 0.05))
print(res_cox1.likelihood_ratio_test(res_Model2_ASC, 0.05))

LRTuple(message='H0 can be rejected at level 5.0%', statistic=49.127263927994136, threshold=7.814727903251179)
LRTuple(message='H0 can be rejected at level 5.0%', statistic=534.4566876591689, threshold=7.814727903251179)


In this case both model are rejected and so, as we said, we need to keep looking for other models

In [41]:
v_w_cox2 = v_walk_Model1  
v_c_cox2 = v_cycling_Model1
v_pt_cox2 = v_PT_Model1 + beta_cost_interacted * cost_transit
v_driving_cox2 = v_driving_Model1 + beta_cost_interacted * cost_driving

In [42]:
V_cox2 = {1: v_w_cox2, 2: v_c_cox2, 3: v_pt_cox2, 4: v_driving_cox2}
logprob_cox2 = models.loglogit(V_cox2, None, travel_mode)
biogeme_cox2 = bio.BIOGEME(database, logprob_cox2)
biogeme_cox2.modelName = 'logit_cox2'
res_cox2 = biogeme_cox2.estimate()

In [43]:
print(res_cox2.printGeneralStatistics())

Number of estimated parameters:	9
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4371.267
Final log likelihood:	-4371.267
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00206
Akaike Information Criterion:	8760.533
Bayesian Information Criterion:	8819.188
Final gradient norm:	8.6372E-04
Nbr of threads:	64



In [44]:
res_cox2.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost,-0.194854,0.033653,-5.790010,7.038225e-09
beta_cost_int,0.147775,0.079102,1.868163,6.173939e-02
beta_time_cycling,-5.450925,0.493378,-11.048165,0.000000e+00
beta_time_driving,-5.797144,0.384984,-15.058141,0.000000e+00
beta_time_pt,-3.124660,0.244302,-12.790159,0.000000e+00
beta_time_walking,-8.165163,0.386227,-21.140848,0.000000e+00
constant_cycling,-2.559675,0.150780,-16.976168,0.000000e+00
constant_pt,-0.406721,0.069511,-5.851144,4.882045e-09
constant_walking,1.933683,0.129094,14.978860,0.000000e+00


In [45]:
print(res_cox2.likelihood_ratio_test(res_Model1, 0.05))
print(res_cox2.likelihood_ratio_test(res_Model2_cost, 0.05))

LRTuple(message='H0 cannot be rejected at level 5.0%', statistic=3.288482000320073, threshold=3.841458820694124)
LRTuple(message='H0 can be rejected at level 5.0%', statistic=32.08931386668519, threshold=3.841458820694124)


Preferred model is Model 1

# Model 3: #

We would like to apply the boxcox transofrmation to the time variables. Our choice is based on the fact that we think it's more likely to change the perception of time in a non linear way, when the duration of a travel increases, especially in context such as walking or cycling. Anyway, since we want to be sure that our assumption does not bias the model too much, we are going to apply the boxcox transformation which depending on lambda parameter will give us a wider range of correlation between the time and its perception. 

In [46]:
lambda_boxcox_w = Beta('lambda_boxcox_w', 1, -10, 10, 0)
lambda_boxcox_c = Beta('lambda_boxcox_c', 1, -10, 10, 0)
lambda_boxcox_pt = Beta('lambda_boxcox_pt', 1, -10, 10, 0)
lambda_boxcox_driving = Beta('lambda_boxcox_driving', 1, -10, 10, 0)
boxcox_time_w = models.boxcox(dur_walking, lambda_boxcox_w)
boxcox_time_c = models.boxcox(dur_cycling, lambda_boxcox_c)
boxcox_time_pt = models.boxcox(dur_pt, lambda_boxcox_pt)
boxcox_time_driving = models.boxcox(dur_driving, lambda_boxcox_driving)

In [47]:
v_w_Model3 = (
    constant_walking
    + beta_time_walking * boxcox_time_w
)

v_c_Model3 = (
    constant_cycling
    + beta_time_cycling * boxcox_time_c
)

v_pt_Model3= (
    constant_pt
    + beta_cost * cost_transit    
    + beta_time_pt * boxcox_time_pt
)

v_driving_Model3 = (
    beta_cost * cost_driving
    + beta_time_driving * boxcox_time_driving
)

In [48]:
v_Model3 = {1: v_w_Model3, 2: v_c_Model3, 3: v_pt_Model3, 4: v_driving_Model3}
logprob_Model3 = models.loglogit(v_Model3, None, travel_mode)
biogeme_Model3 = bio.BIOGEME(database, logprob_Model3)
biogeme_Model3.modelName = 'logit_Model3'
res_Model3 = biogeme_Model3.estimate()


In [49]:
print(res_Model3.printGeneralStatistics())

Number of estimated parameters:	12
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4277.532
Final log likelihood:	-4277.532
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00281
Akaike Information Criterion:	8579.064
Bayesian Information Criterion:	8657.271
Final gradient norm:	2.7665E-02
Nbr of threads:	64



In [50]:
res_Model3.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost,-0.121931,0.013848,-8.805163,0.000000e+00
beta_time_cycling,-3.219416,0.508661,-6.329198,2.464391e-10
beta_time_driving,-3.376682,0.296580,-11.385409,0.000000e+00
beta_time_pt,-3.551234,0.337464,-10.523293,0.000000e+00
beta_time_walking,-5.256316,0.379644,-13.845368,0.000000e+00
constant_cycling,-2.329929,0.260703,-8.937104,0.000000e+00
constant_pt,1.539500,0.140732,10.939190,0.000000e+00
constant_walking,0.099235,0.282442,0.351345,7.253293e-01
lambda_boxcox_c,0.279432,0.129578,2.156483,3.104598e-02
lambda_boxcox_driving,0.320108,0.057551,5.562126,2.665073e-08


The estiamted lambdas are quite different, and this leaves us happy, especially the lambda_boxcox_pt is close to 1 that means it's close to be linear (whihc we expected since on public transport you may not feel too much difference between the first minutes and the last ones, since as we've already said you're able to rest or to do other stuffs). Another interesting fact we can notice is that all lambdas are smaller than 1 which means that the more we increase the time the less impact we get on the individual. The signs of the betas are still coherent and the order of ASCs reamins as usual. Looking at t-test values, we see that all of them are gratetr in absoulte value than 1.96, except for ASC_walking, which results to be quite low meaning that this is not really significant.

In [51]:
res_Model3.likelihood_ratio_test(res_Model1, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=190.7572699829525, threshold=9.487729036781154)

We applied the likelihood ratio test since boxcox transformation can be resricted to model 1 (linear) by putting all the lambdas to 1. The result seems good and so now our preferred model in Model3, this confirms our intial assumptions about different perception of time.

# Model 4: #

We choose to test a nested structure. Divding our options in MOTORIZED and UNMOTORIZED (fare grafi su latex). The division seems reasonable to us since when you have to decide a travel mode, at least in our case, the flow of thoughts usually starts from deciding if we want to go with a motorized or unmotorized alternative, depending on how fast do we have to be, on how do we phisically fell and on other factors such as weather conditions. We also notice that this division can be interpreted as a split between FREE and PAID alternatives, which will give us the possibility to interpret the reuslts in a dual way.

In [52]:
mu_motorized = Beta('mu_motorized', 1, 0, None, 0)
mu_unmotorized = Beta('mu_unmotorized', 1, 0, None, 0)
unmotorized = mu_unmotorized, [1, 2]
motorized = mu_motorized, [3, 4]
nests = unmotorized, motorized
logprob_Model4 = models.lognested(v_Model3, None, nests, travel_mode) 
biogeme_Model4 = bio.BIOGEME(database, logprob_Model4)
biogeme_Model4.modelName = 'nested_logit_Model4'
res_Model4 = biogeme_Model4.estimate()


In [53]:
res_Model4.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost,-0.095513,0.020288,-4.707939,2.502342e-06
beta_time_cycling,-4.195616,1.134928,-3.696813,2.183231e-04
beta_time_driving,-2.569629,0.541272,-4.747389,2.060589e-06
beta_time_pt,-2.784873,0.552464,-5.040820,4.635424e-07
beta_time_walking,-6.107417,0.570887,-10.698116,0.000000e+00
constant_cycling,-2.780108,0.390406,-7.121066,1.070921e-12
constant_pt,1.190504,0.239824,4.964064,6.903331e-07
constant_walking,-1.634590,0.593654,-2.753439,5.897275e-03
lambda_boxcox_c,1.556361,0.556275,2.797828,5.144752e-03
lambda_boxcox_driving,0.258577,0.067901,3.808145,1.400134e-04


Since the estimation for mu_unmotorized is less than one, this nested logit model is not valid, so for the sake of a good comparison with model 3, we are gonna try some other split, hoping to find a valid one. The next try will be about splitting the altenratives in PUBLIC and PRIVATE

In [54]:
mu_private = Beta('mu_private', 1, 0, None, 0)
private = mu_private, [1,2,4]
public = 1.0,[3]
nests = private, public
logprob_Model4_v2 = models.lognested(v_Model3, None, nests, travel_mode)
biogeme_Model4_v2 = bio.BIOGEME(database, logprob_Model4_v2)
biogeme_Model4_v2.modelName = 'nested_logit_Model4_v2'
res_Model4_v2 = biogeme_Model4_v2.estimate()

In [55]:
res_Model4_v2.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost,-0.122300,0.014391,-8.498556,0.000000e+00
beta_time_cycling,-3.215516,0.518201,-6.205149,5.464502e-10
beta_time_driving,-3.371131,0.304441,-11.073174,0.000000e+00
beta_time_pt,-3.545232,0.348346,-10.177320,0.000000e+00
beta_time_walking,-5.285215,0.479692,-11.017934,0.000000e+00
constant_cycling,-2.354015,0.346229,-6.799007,1.053424e-11
constant_pt,1.538232,0.141445,10.875124,0.000000e+00
constant_walking,0.070206,0.413226,0.169897,8.650909e-01
lambda_boxcox_c,0.277023,0.135386,2.046170,4.073969e-02
lambda_boxcox_driving,0.317848,0.063035,5.042450,4.596089e-07


We notice that we did not put any mu_public since there is just one alternative is this nest. Anyway the estimation of mu_private is still less than 1 so we wanna keep on with our exploration, trying to exploit a cross nested model, where the split is between PRIVATE and MOTORIZED alternatives, where CAR is common to both the nests.

In [56]:
mu_motorized = Beta('mu_motorized', 1, 1, None, 0)
mu_private = Beta('mu_private', 1, 1, None, 0)
alpha_motorized_car = Beta('alpha_motorized_car', 0.5, 0, 1, 0)
alpha_private_car = 1 - alpha_motorized_car
alpha_motorized = {
    1 : 0.0,
    2 : 0.0,
    3 : 1.0,
    4 : alpha_motorized_car
}
alpha_private = {
    1 : 1.0,
    2 : 1.0,
    3 : 0.0,
    4 : alpha_private_car
}

nest_motorized = mu_motorized, alpha_motorized
nest_private = mu_private, alpha_private

nests = nest_motorized, nest_private

logprob_Model4_v3 = models.logcnl_avail(v_Model3, None, nests, travel_mode)
biogeme_Model4_v3 = bio.BIOGEME(database, logprob_Model4_v3)
biogeme_Model4_v3.modelName = 'cnl_avail_Model4_v3'
res_Model4_v3 = biogeme_Model4_v3.estimate()


In [57]:
print(res_Model4_v3.printGeneralStatistics())

Number of estimated parameters:	15
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4270.487
Final log likelihood:	-4270.487
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00351
Akaike Information Criterion:	8570.973
Bayesian Information Criterion:	8668.731
Final gradient norm:	1.7004E-02
Nbr of threads:	64



In [58]:
res_Model4_v3.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
alpha_motorized_car,0.476319,0.099565,4.784014,1.718285e-06
beta_cost,-0.090862,0.014995,-6.059643,1.364242e-09
beta_time_cycling,-2.330014,0.501557,-4.645563,3.391510e-06
beta_time_driving,-2.318131,0.352254,-6.580842,4.677903e-11
beta_time_pt,-2.306788,0.366268,-6.298092,3.013321e-10
beta_time_walking,-4.126894,0.483776,-8.530581,0.000000e+00
constant_cycling,-2.485102,0.310523,-8.002947,1.110223e-15
constant_pt,0.964056,0.166274,5.798003,6.710912e-09
constant_walking,-0.769839,0.439394,-1.752048,7.976561e-02
lambda_boxcox_c,0.254218,0.155802,1.631679,1.027471e-01


The estimation now leaves us satisfied, both because the theoretical constraints are respected (mu_motorized and mu_private are both grater than 1) and, aboveall, the values are reasonable. In particular, the fact that alpha is estimated at 0.476 is in line with our expectations since we think that there is no particular dominance between the two nests regarding the belonging of CAR alternative. So now it's time to test our model in order to determine if this version is better than our actual preferred one. We apply a logliekelihood ratio test.

In [59]:
res_Model4_v3.likelihood_ratio_test(res_Model3, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=14.090894662507708, threshold=7.814727903251179)

So now the preferred model is Model 4, in its last version, where we propose a division between PRIVATE and MOTORIZED alternatives, with CAR belonging to both nests.

# Market shares: #

In [60]:
condition1 = df['age'] >= 45  
condition2 = df['female'] == 1
condition3 = df['age'] < 45
condition4 = df['female'] == 0  

size_S1 = (condition2 & condition3).sum()
size_S2 = (condition2 & condition1).sum()
size_S3 = (condition4 & condition3).sum()
size_S4 = (condition4 & condition1).sum()
S = size_S1 + size_S2 + size_S3 + size_S4
print("Size of S1: ", size_S1, "\nSize of S2: ", size_S2, "\nSize of S3: ", size_S3, "\nSize of S4: ", size_S4)
print(S)

Size of S1:  1663 
Size of S2:  1004 
Size of S3:  1433 
Size of S4:  900
5000


In [61]:
P1 = 2841376
P2 = 1519948
P3 = 2926408
P4 = 1379198
P = P1 + P2 + P3 + P4

w1 = (P1/P)/(size_S1/S)
w2 = (P2/P)/(size_S2/S)
w3 = (P3/P)/(size_S3/S)
w4 = (P4/P)/(size_S4/S)
print("weight of S1: ", w1, "\nweight of S2: ", w2, "\nweight of S3: ", w3, "\nweight of S4: ", w4)

weight of S1:  0.9856918689022015 
weight of S2:  0.8733729419061216 
weight of S3:  1.1781305028128637 
weight of S4:  0.8840744197900654


In [62]:
filters = {
    'female_44_less' : condition2 & condition3,
    'female_44_more' : condition2 & condition1,
    'male_44_less' : condition4 & condition3,
    'male_44_more' : condition4 & condition1
}

In [63]:
weights = {
    'female_44_less' : w1,
    'female_44_more' : w2,
    'male_44_less' : w3,
    'male_44_more' : w4
}

for k,f in filters.items():
    df.loc[f, 'Weight'] = weights[k]

In [64]:
df

,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,...,dur_pt_bus,dur_pt_int,pt_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_ccharge,driving_traffic_percent,cost_driving,Weight
0,1,0,0,1,4,3,1,1,1.0,1,...,0.055556,0.000000,0,0.059444,1.5,0.15,0.0,0.112150,0.15,0.873373
1,13,1,1,1,4,3,1,5,0.0,1,...,0.122222,0.000000,0,0.132222,0.0,0.50,0.0,0.065126,0.50,0.884074
2,19,4,0,1,3,3,6,5,0.0,1,...,0.312222,0.000000,0,0.221667,0.0,0.56,0.0,0.086466,0.56,0.873373
3,20,5,1,0,4,3,1,5,0.0,1,...,0.062222,0.000000,0,0.117222,0.0,0.41,0.0,0.097156,0.41,0.873373
4,39,9,2,0,4,3,1,5,0.0,1,...,0.225000,0.000000,0,0.200833,0.0,0.48,0.0,0.378976,0.48,0.985692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,81035,17603,1,1,3,1,1,1,1.0,3,...,0.000000,0.000000,0,0.392222,2.4,0.98,0.0,0.547450,0.98,0.884074
4996,81040,17604,2,0,3,1,1,2,0.0,3,...,0.353333,0.176667,1,0.288889,0.0,0.85,0.0,0.175000,0.85,1.178131
4997,81045,17605,0,1,4,3,1,5,0.0,3,...,0.045833,0.000000,0,0.067778,0.0,0.17,0.0,0.024590,0.17,0.873373
4998,81066,17608,0,3,3,3,6,1,1.0,3,...,0.389444,0.000000,0,0.193333,1.5,0.61,0.0,0.485632,0.61,0.985692


In [65]:
sum(df['Weight'])

4999.999999999712

In [66]:
database_weight = db.Database('lpmc_2', df)

In [67]:
Weight = Variable('Weight')
prob_walking = models.cnl_avail(v_Model3, None, nests, 1)
prob_cycling = models.cnl_avail(v_Model3, None, nests, 2)
prob_pt = models.cnl_avail(v_Model3, None, nests, 3)
prob_driving = models.cnl_avail(v_Model3, None, nests, 4)

simulate = {
    'Weight': Weight,
    'Prob. walking': prob_walking,
    'Prob. cycling': prob_cycling,
    'Prob. public transport': prob_pt,
    'Prob. driving': prob_driving
}

biosim = bio.BIOGEME(database_weight, simulate)
simulated_values = biosim.simulate(res_Model4_v3.getBetaValues())


In [68]:
simulated_values

,Weight,Prob. walking,Prob. cycling,Prob. public transport,Prob. driving
0,0.873373,0.602341,0.027869,0.066058,0.303731
1,0.884074,0.014402,0.023152,0.261245,0.701202
2,0.873373,0.047595,0.064701,0.308853,0.578850
3,0.873373,0.058296,0.032773,0.139061,0.769870
4,0.985692,0.211380,0.064820,0.353804,0.369996
...,...,...,...,...,...
4995,0.884074,0.008484,0.025398,0.624870,0.341247
4996,1.178131,0.014393,0.038959,0.236167,0.710481
4997,0.873373,0.500434,0.029927,0.116679,0.352961
4998,0.985692,0.011595,0.020540,0.267151,0.700714


In [69]:
simulated_values['Weighted Prob. walking'] = simulated_values['Weight'] * simulated_values['Prob. walking']
simulated_values['Weighted Prob. cycling'] = simulated_values['Weight'] * simulated_values['Prob. cycling']
simulated_values['Weighted Prob. public transport'] = simulated_values['Weight'] * simulated_values['Prob. public transport']
simulated_values['Weighted Prob. driving'] = simulated_values['Weight'] * simulated_values['Prob. driving']

market_share_walking = simulated_values['Weighted Prob. walking'].mean()
market_share_cycling = simulated_values['Weighted Prob. cycling'].mean()
market_share_pt = simulated_values['Weighted Prob. public transport'].mean()
market_share_driving = simulated_values['Weighted Prob. driving'].mean()

print("Market share of walking: ", market_share_walking, "\nMarket share of cycling: ", market_share_cycling, "\nMarket share of public transport: ", market_share_pt, "\nMarket share of driving: ", market_share_driving)
print(market_share_cycling + market_share_driving + market_share_pt + market_share_walking)

Market share of walking:  0.17100866022436484 
Market share of cycling:  0.03190479632397999 
Market share of public transport:  0.36380676271959894 
Market share of driving:  0.43327978073205614
0.9999999999999998


In [70]:
# biogeme_Model4_v3.bootstrap_samples = 100
# results_bootstrap = biogeme_Model4_v3.estimate(run_bootstrap=True)


In [71]:
# betas = biogeme_Model4_v3.freeBetaNames()
# b = results_bootstrap.getBetasForSensitivityAnalysis(betas)
# left, right = biosim.confidenceIntervals(b, 0.9)

In [72]:
# left['weighted Prob. walking'] = left['Weight'] * left['Prob. walking']
# left['weighted Prob. cycling'] = left['Weight'] * left['Prob. cycling']
# left['weighted Prob. public transport'] = left['Weight'] * left['Prob. public transport']
# left['weighted Prob. driving'] = left['Weight'] * left['Prob. driving']

# right['weighted Prob. walking'] = right['Weight'] * right['Prob. walking']
# right['weighted Prob. cycling'] = right['Weight'] * right['Prob. cycling']
# right['weighted Prob. public transport'] = right['Weight'] * right['Prob. public transport']
# right['weighted Prob. driving'] = right['Weight'] * right['Prob. driving']

# left_market_share_walking = left['weighted Prob. walking'].mean()
# left_market_share_cycling = left['weighted Prob. cycling'].mean()
# left_market_share_pt = left['weighted Prob. public transport'].mean()
# left_market_share_driving = left['weighted Prob. driving'].mean()

# right_market_share_walking = right['weighted Prob. walking'].mean()
# right_market_share_cycling = right['weighted Prob. cycling'].mean()
# right_market_share_pt = right['weighted Prob. public transport'].mean()
# right_market_share_driving = right['weighted Prob. driving'].mean()



In [73]:
# print('CI for market share of walking: [', left_market_share_walking, right_market_share_walking, ']')
# print('CI for market share of cycling: [', left_market_share_cycling, right_market_share_cycling, ']')
# print('CI for market share of public transport: [', left_market_share_pt, right_market_share_pt, ']')
# print('CI for market share of driving: [', left_market_share_driving, right_market_share_driving, ']')

The market shares are in line with our expectetions. Car is the most used alternative, which is reasonable in a big metropolitan city like London. Public transport is the 2nd alternative and it makes sense, as it makes sense that slow modes have a minority market share, in particular the market share for cycling is quite low, but it's reasonable considering the typical weather and traffic condtion of London.

In [74]:
df[df['travel_mode'] == 1]['Weight']

6       1.178131
9       0.985692
13      1.178131
14      1.178131
20      0.873373
          ...   
4954    0.873373
4958    0.985692
4960    0.985692
4988    0.985692
4993    0.985692
Name: Weight, Length: 857, dtype: float64

In [75]:
#compute market shares based on the dataset
data_market_share_walking = df[df['travel_mode'] == 1]['Weight'].sum() / len(df)
print("Data market share of walking: ", data_market_share_walking)
data_market_share_cycling = df[df['travel_mode'] == 2]['Weight'].sum() / len(df)
print("Data market share of cycling: ", data_market_share_cycling)
data_market_share_pt = df[df['travel_mode'] == 3]['Weight'].sum() / len(df)
print("Data market share of public transport: ", data_market_share_pt)
data_market_share_driving = df[df['travel_mode'] == 4]['Weight'].sum() / len(df)
print('Data market share of driving: ', data_market_share_driving)


Data market share of walking:  0.1729534942415637
Data market share of cycling:  0.03275558060007598
Data market share of public transport:  0.3650968121057645
Data market share of driving:  0.4291941130525958


We notice how much similar the real weighted market shares are to the predicted ones.

# Forecasting: #

### Scenario 1


In [76]:
# we create a new database with the same characteristics as the original one but modifying the cost of driving according to our scenario
df_s1 = df.copy()
df_s1['cost_driving_ccharge'] = df_s1['cost_driving_ccharge'] + 1.5
database_s1 = db.Database('lpmc_s1', df_s1)


In [77]:
Weight = Variable('Weight')
prob_walking_s1 = models.cnl_avail(v_Model3, None, nests, 1)
prob_cycling_s1 = models.cnl_avail(v_Model3, None, nests, 2)
prob_pt_s1 = models.cnl_avail(v_Model3, None, nests, 3)
prob_driving_s1 = models.cnl_avail(v_Model3, None, nests, 4)

simulate_s1 = {
    'Weight': Weight,
    'Prob. walking': prob_walking_s1,
    'Prob. cycling': prob_cycling_s1,
    'Prob. public transport': prob_pt_s1,
    'Prob. driving': prob_driving_s1,
}

biosim_s1 = bio.BIOGEME(database_s1, simulate_s1)
simulated_values_s1 = biosim_s1.simulate(res_Model4_v3.getBetaValues())

In [78]:
simulated_values_s1['Weighted Prob. walking'] = simulated_values_s1['Weight'] * simulated_values_s1['Prob. walking']
simulated_values_s1['Weighted Prob. cycling'] = simulated_values_s1['Weight'] * simulated_values_s1['Prob. cycling']
simulated_values_s1['Weighted Prob. public transport'] = simulated_values_s1['Weight'] * simulated_values_s1['Prob. public transport']
simulated_values_s1['Weighted Prob. driving'] = simulated_values_s1['Weight'] * simulated_values_s1['Prob. driving']


market_share_walking_s1 = simulated_values_s1['Weighted Prob. walking'].mean()
market_share_cycling_s1 = simulated_values_s1['Weighted Prob. cycling'].mean()
market_share_pt_s1 = simulated_values_s1['Weighted Prob. public transport'].mean()
market_share_driving_s1 = simulated_values_s1['Weighted Prob. driving'].mean()

print("Market share of walking: ", market_share_walking_s1, "\nMarket share of cycling: ", market_share_cycling_s1, "\nMarket share of public transport: ", market_share_pt_s1, "\nMarket share of driving: ", market_share_driving_s1)
print(market_share_cycling_s1 + market_share_driving_s1 + market_share_pt_s1 + market_share_walking_s1)


Market share of walking:  0.17100866022436484 
Market share of cycling:  0.03190479632397999 
Market share of public transport:  0.36380676271959894 
Market share of driving:  0.43327978073205614
0.9999999999999998


### Scenario 2

In [79]:
df_s2 = df.copy()
df_s2['cost_transit'] = df_s2['cost_transit'] * 0.8
database_s2 = db.Database('lpmc_s2', df_s2)

In [80]:
Weight = Variable('Weight')
prob_walking_s2 = models.cnl_avail(v_Model3, None, nests, 1)
prob_cycling_s2 = models.cnl_avail(v_Model3, None, nests, 2)
prob_pt_s2 = models.cnl_avail(v_Model3, None, nests, 3)
prob_driving_s2 = models.cnl_avail(v_Model3, None, nests, 4)

simulate_s2 = {
    'Weight': Weight,
    'Prob. walking': prob_walking_s2,
    'Prob. cycling': prob_cycling_s2,
    'Prob. public transport': prob_pt_s2,
    'Prob. driving': prob_driving_s2
}

biosim_s2 = bio.BIOGEME(database_s2, simulate_s2)
simulated_values_s2 = biosim_s2.simulate(res_Model4_v3.getBetaValues())



simulated_values_s2['Weighted Prob. walking'] = simulated_values_s2['Weight'] * simulated_values_s2['Prob. walking']
simulated_values_s2['Weighted Prob. cycling'] = simulated_values_s2['Weight'] * simulated_values_s2['Prob. cycling']
simulated_values_s2['Weighted Prob. public transport'] = simulated_values_s2['Weight'] * simulated_values_s2['Prob. public transport']
simulated_values_s2['Weighted Prob. driving'] = simulated_values_s2['Weight'] * simulated_values_s2['Prob. driving']

market_share_walking_s2 = simulated_values_s2['Weighted Prob. walking'].mean()
market_share_cycling_s2 = simulated_values_s2['Weighted Prob. cycling'].mean()
market_share_pt_s2 = simulated_values_s2['Weighted Prob. public transport'].mean()
market_share_driving_s2 = simulated_values_s2['Weighted Prob. driving'].mean()

print("Market share of walking: ", market_share_walking_s2, "\nMarket share of cycling: ", market_share_cycling_s2, "\nMarket share of public transport: ", market_share_pt_s2, "\nMarket share of driving: ", market_share_driving_s2)
print(market_share_cycling_s2 + market_share_driving_s2 + market_share_pt_s2 + market_share_walking_s2)


Market share of walking:  0.17049930771890937 
Market share of cycling:  0.0315439110278445 
Market share of public transport:  0.37127493644825554 
Market share of driving:  0.42668184480499055
1.0


The market shares of both the scenarios reflect our expetations: in the first case we see a decreasing of market shares of the car due to the increasing of the cost of the car and in the second case we see an increasing in market shares of public transport due to the decreasing in the cost.

The most effective scenario to decrease the share of car is the first one when we charge an addtional cost for the car users. This is reasonable for us since we think that somehow when an user undergoes a price increase in the choice he's currently using he is more likely to move to other alternatives rather than when the price of his alternative remains the same and another one decreases.

### PT Revenues Scenario 1

In [81]:

Weight = Variable('Weight')
prob_walking_s1_R = models.cnl_avail(v_Model3, None, nests, 1)
prob_cycling_s1_R = models.cnl_avail(v_Model3, None, nests, 2)
prob_pt_s1_R = models.cnl_avail(v_Model3, None, nests, 3)
prob_driving_s1_R = models.cnl_avail(v_Model3, None, nests, 4)
simulate_s1_R = {
    'Weight': Weight,
    'Revenue PT': prob_pt_s1 * cost_transit,
}

biosim_s1_R = bio.BIOGEME(database_s1, simulate_s1_R)
simulated_values_s1_R = biosim_s1_R.simulate(res_Model4_v3.getBetaValues())




In [82]:
simulated_values_s1_R['Weighted Revenue PT'] = simulated_values_s1_R['Weight'] * simulated_values_s1_R['Revenue PT']
simulated_values_s1_R['Weighted Revenue PT'].sum()

3597.7302882303566

### PT Revenues Scenario 2

In [83]:
Weight = Variable('Weight')
prob_walking_s2_R = models.cnl_avail(v_Model3, None, nests, 1)
prob_cycling_s2_R = models.cnl_avail(v_Model3, None, nests, 2)
prob_pt_s2_R = models.cnl_avail(v_Model3, None, nests, 3)
prob_driving_s2_R = models.cnl_avail(v_Model3, None, nests, 4)
simulate_s2_R = {
    'Weight': Weight,
    'Revenue PT': prob_pt_s2 * cost_transit ,
}

biosim_s2_R = bio.BIOGEME(database_s2, simulate_s2_R)
simulated_values_s2_R = biosim_s2_R.simulate(res_Model4_v3.getBetaValues())



In [84]:
simulated_values_s2_R['Weighted Revenue PT'] = simulated_values_s2_R['Weight'] * simulated_values_s2_R['Revenue PT']
simulated_values_s2_R['Weighted Revenue PT'].sum()

2965.0626717256237

### PT Revenues base scenario

In [85]:
Weight = Variable('Weight')
prob_walking_R = models.cnl_avail(v_Model3, None, nests, 1)
prob_cycling_R = models.cnl_avail(v_Model3, None, nests, 2)
prob_pt_R = models.cnl_avail(v_Model3, None, nests, 3)
prob_driving_R = models.cnl_avail(v_Model3, None, nests, 4)
simulate_R = {
    'Weight': Weight,
    'Revenue PT': prob_pt_R * cost_transit,
}

biosim_R = bio.BIOGEME(database_weight, simulate_R)
simulated_values_R = biosim_R.simulate(res_Model4_v3.getBetaValues())


In [86]:
simulated_values_R['Weighted Revenue PT'] = simulated_values_R['Weight'] * simulated_values_R['Revenue PT']
simulated_values_R['Weighted Revenue PT'].sum()

3597.7302882303566

The scenario that maximizes the PT total revenue is scenario 1. This is due to the fact that this scenario increases the market shares of around 3% without modifying PT cost. On the other hand the second scenario increases the market shares of PT of a really low percentage and it moreover decreases the price, so a minor revenue is inevitable. Scenario 1 also outperforms the base case since as we said we have an increase of the market shares without a modification of the price.

## Value of time

### Car

In [87]:
btd = res_Model4_v3.getBetaValues()['beta_time_driving']
bc = res_Model4_v3.getBetaValues()['beta_cost']
lambda_boxcox_driving = res_Model4_v3.getBetaValues()['lambda_boxcox_driving']

VOT_car = ( btd / bc * (df['dur_driving'] ** (lambda_boxcox_driving - 1))).mean()
VOT_car

87.05656409223022

In [88]:
btpt = res_Model4_v3.getBetaValues()['beta_time_pt']
lambda_boxcox_pt = res_Model4_v3.getBetaValues()['lambda_boxcox_pt']
dur_pt = df['dur_pt_access'] + df['dur_pt_rail'] + df['dur_pt_bus'] + df['dur_pt_int']

VOT_pt = ( btpt / bc * (dur_pt ** (lambda_boxcox_pt - 1))).mean()
VOT_pt

32.789617176319766

In [89]:
from biogeme.expressions import Derive

In [92]:
direct_elas_driving_cost = Derive(prob_driving, 'cost_driving') * cost_driving / prob_driving
direct_elas_pt_cost = Derive(prob_pt, 'cost_transit') * cost_transit / prob_pt
cross_elas_pt_carcost = Derive(prob_pt, 'cost_driving') * cost_driving / prob_pt
cross_elas_cycling_carcost = Derive(prob_cycling, 'cost_driving') * cost_driving / prob_cycling
cross_elas_walking_carcost = Derive(prob_walking, 'cost_driving') * cost_driving / prob_walking
cross_elas_car_ptcost = Derive(prob_driving, 'cost_transit') * cost_transit / prob_driving
cross_elas_cycling_ptcost = Derive(prob_cycling, 'cost_transit') * cost_transit / prob_cycling
cross_elas_walking_ptcost = Derive(prob_walking, 'cost_transit') * cost_transit / prob_walking

In [93]:
simulate_elas = {
    'Weight': Weight,
    'Direct elasticity driving cost': direct_elas_driving_cost,
    'Direct elasticity PT cost': direct_elas_pt_cost,
    'Cross elasticity PT car cost': cross_elas_pt_carcost,
    'Cross elasticity cycling car cost': cross_elas_cycling_carcost,
    'Cross elasticity walking car cost': cross_elas_walking_carcost,
    'Cross elasticity car PT cost': cross_elas_car_ptcost,
    'Cross elasticity cycling PT cost': cross_elas_cycling_ptcost,
    'Cross elasticity walking PT cost': cross_elas_walking_ptcost,
    'prob public transport': prob_pt,
    'prob cycling': prob_cycling,
    'prob walking': prob_walking,
    'prob driving': prob_driving
}

In [94]:
biosim_elas = bio.BIOGEME(database_weight, simulate_elas)
simulated_values_elas = biosim_elas.simulate(res_Model4_v3.getBetaValues())


In [95]:
simulated_values_elas

,Weight,Direct elasticity driving cost,Direct elasticity PT cost,Cross elasticity PT car cost,Cross elasticity cycling car cost,Cross elasticity walking car cost,Cross elasticity car PT cost,Cross elasticity cycling PT cost,Cross elasticity walking PT cost,prob public transport,prob cycling,prob walking,prob driving
0,0.873373,-0.013180,-0.243357,0.015746,0.004701,0.004701,0.034247,0.009003,0.009003,0.066058,0.027869,0.602341,0.303731
1,0.884074,-0.024634,0.000000,0.060269,0.040692,0.040692,0.000000,0.000000,0.000000,0.261245,0.023152,0.014402,0.701202
2,0.873373,-0.036585,0.000000,0.054864,0.037690,0.037690,0.000000,0.000000,0.000000,0.308853,0.064701,0.047595,0.578850
3,0.873373,-0.015171,0.000000,0.060942,0.035192,0.035192,0.000000,0.000000,0.000000,0.139061,0.032773,0.058296,0.769870
4,0.985692,-0.043789,0.000000,0.029785,0.020506,0.020506,0.000000,0.000000,0.000000,0.353804,0.064820,0.211380,0.369996
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.884074,-0.080832,-0.109237,0.041588,0.047120,0.047120,0.186499,0.136265,0.136265,0.624870,0.025398,0.008484,0.341247
4996,1.178131,-0.040599,0.000000,0.106460,0.069394,0.069394,0.000000,0.000000,0.000000,0.236167,0.038959,0.014393,0.710481
4997,0.873373,-0.014928,0.000000,0.016449,0.006316,0.006316,0.000000,0.000000,0.000000,0.116679,0.029927,0.500434,0.352961
4998,0.985692,-0.030116,-0.183906,0.073007,0.049746,0.049746,0.068445,0.036411,0.036411,0.267151,0.020540,0.011595,0.700714


In [96]:
simulated_values_elas['Numerator direct elasticity driving cost'] = simulated_values_elas['Weight'] * simulated_values_elas['Direct elasticity driving cost'] * simulated_values_elas['prob driving']
simulated_values_elas['Numerator direct elasticity PT cost'] = simulated_values_elas['Weight'] * simulated_values_elas['Direct elasticity PT cost'] * simulated_values_elas['prob public transport']
simulated_values_elas['Numerator cross elasticity PT car cost'] = simulated_values_elas['Weight'] * simulated_values_elas['Cross elasticity PT car cost'] * simulated_values_elas['prob public transport']
simulated_values_elas['Numerator cross elasticity cycling car cost'] = simulated_values_elas['Weight'] * simulated_values_elas['Cross elasticity cycling car cost'] * simulated_values_elas['prob cycling']
simulated_values_elas['Numerator cross elasticity walking car cost'] = simulated_values_elas['Weight'] * simulated_values_elas['Cross elasticity walking car cost'] * simulated_values_elas['prob walking']
simulated_values_elas['Numerator cross elasticity car PT cost'] = simulated_values_elas['Weight'] * simulated_values_elas['Cross elasticity car PT cost'] * simulated_values_elas['prob driving']
simulated_values_elas['Numerator cross elasticity cycling PT cost'] = simulated_values_elas['Weight'] * simulated_values_elas['Cross elasticity cycling PT cost'] * simulated_values_elas['prob cycling']
simulated_values_elas['Numerator cross elasticity walking PT cost'] = simulated_values_elas['Weight'] * simulated_values_elas['Cross elasticity walking PT cost'] * simulated_values_elas['prob walking']


In [97]:
simulated_values_elas['Denominator Driving'] = simulated_values_elas['Weight'] * simulated_values_elas['prob driving']
simulated_values_elas['Denominator PT'] = simulated_values_elas['Weight'] * simulated_values_elas['prob public transport']
simulated_values_elas['Denominator cycling'] = simulated_values_elas['Weight'] * simulated_values_elas['prob cycling']
simulated_values_elas['Denominator walking'] = simulated_values_elas['Weight'] * simulated_values_elas['prob walking']

In [98]:
aggregate_direct_elasticity_driving_cost = simulated_values_elas['Numerator direct elasticity driving cost'].sum() / simulated_values_elas['Denominator Driving'].sum()
aggregate_direct_elasticity_PT_cost = simulated_values_elas['Numerator direct elasticity PT cost'].sum() / simulated_values_elas['Denominator PT'].sum()
aggregate_cross_elasticity_PT_carcost = simulated_values_elas['Numerator cross elasticity PT car cost'].sum() / simulated_values_elas['Denominator PT'].sum()
aggregate_cross_elasticity_cycling_carcost = simulated_values_elas['Numerator cross elasticity cycling car cost'].sum() / simulated_values_elas['Denominator cycling'].sum()
aggregate_cross_elasticity_walking_carcost = simulated_values_elas['Numerator cross elasticity walking car cost'].sum() / simulated_values_elas['Denominator walking'].sum()
aggregate_cross_elasticity_car_PTcost = simulated_values_elas['Numerator cross elasticity car PT cost'].sum() / simulated_values_elas['Denominator Driving'].sum()
aggregate_cross_elasticity_cycling_PTcost = simulated_values_elas['Numerator cross elasticity cycling PT cost'].sum() / simulated_values_elas['Denominator cycling'].sum()
aggregate_cross_elasticity_walking_PTcost = simulated_values_elas['Numerator cross elasticity walking PT cost'].sum() / simulated_values_elas['Denominator walking'].sum()


In [101]:
print(
    'Aggregate direct elasticity driving cost: ', aggregate_direct_elasticity_driving_cost*100 , '%',
    '\nAggregate direct elasticity PT cost: ', aggregate_direct_elasticity_PT_cost*100,'%',
    '\nAggregate cross elasticity PT car cost: ', aggregate_cross_elasticity_PT_carcost *100, '%',
    '\nAggregate cross elasticity cycling car cost: ', aggregate_cross_elasticity_cycling_carcost*100, '%',
    '\nAggregate cross elasticity walking car cost: ', aggregate_cross_elasticity_walking_carcost*100, '%',
    '\nAggregate cross elasticity car PT cost: ', aggregate_cross_elasticity_car_PTcost*100, '%',
    '\nAggregate cross elasticity cycling PT cost: ', aggregate_cross_elasticity_cycling_PTcost*100, '%',
    '\nAggregate cross elasticity walking PT cost: ', aggregate_cross_elasticity_walking_PTcost*100, '%'
)

Aggregate direct elasticity driving cost:  -7.564341691012941 % 
Aggregate direct elasticity PT cost:  -10.271191102168547 % 
Aggregate cross elasticity PT car cost:  7.708057590278577 % 
Aggregate cross elasticity cycling car cost:  6.287811186230788 % 
Aggregate cross elasticity walking car cost:  1.5941970126596983 % 
Aggregate cross elasticity car PT cost:  7.625601765264518 % 
Aggregate cross elasticity cycling PT cost:  5.681467868816529 % 
Aggregate cross elasticity walking PT cost:  1.4703562254962637 %
